In [0]:
from keras.layers import Layer
from keras import backend as K

tf = K.tf




class AttentionLayer(Layer):
  def __init__(self, n_cells):
    super(AttentionLayer, self).__init__()
    self.n_cells = n_cells
    
    #weights
    self.w1 = self.add_weight(name = "weight_context",
                             shape = (n_cells, 1),
                             trainable = True,
                             initializer = tf.random_uniform_initializer())
    self.w2 = self.add_weight(name = "weight_hidden",
                             shape = (n_cells, 1),
                             trainable = True,
                             initializer = tf.random_uniform_initializer())
    
  def call(self, inputs):
    hiddens, context = inputs, inputs[-1]
    
    hw = tf.matmul(hiddens, self.w2)
    cw = tf.matmul(context, self.w1)
    m = tf.nn.tanh(hw+cw)
    
    s = tf.nn.softmax(m, axis = 1) #please do not remove axis = 1, check documentation for this function
    e = s*hiddens
    
    z = tf.reduce_sum(e, axis = 1)
    z = tf.expand_dims(z, axis = -1)
    print(s.shape, z.shape)
    
    
    return [z, s]
  
  def compute_output_shape(self, input_shape):
    return [(None, self.n_cells, 1), (None, self.n_cells, 1)]

In [1]:
from keras import backend as K
from keras.layers import Input
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Activation

from keras.datasets import mnist

import numpy as np
from keras.utils import np_utils



#loading and preprocessing the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0


y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)


#creating the model

layer_input = Input(x_train.shape[1:])

layer_hidden = LSTM(128, return_sequences = True)(layer_input)
layer_hidden = Activation('relu')(layer_hidden)
layer_hidden = Dropout(0.2)(layer_hidden)

layer_hidden, attention_weights = AttentionLayer(128)(layer_hidden)

layer_hidden = LSTM(128)(layer_hidden)
layer_hidden = Activation('relu')(layer_hidden)
layer_hidden = Dropout(0.2)(layer_hidden)

layer_hidden = Dense(64, activation='relu')(layer_hidden)
layer_hidden = Dropout(0.2)(layer_hidden)

layer_output = Dense(10, activation='softmax')(layer_hidden)

model = Model(layer_input, layer_output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


#training the model

m_loss = model.fit(x_train, y_train, epochs = 3, batch_size = 32, validation_data=(x_test, y_test))


#testing the results


#creating a model to get the attention weights for the given test values.
a_model = Model(layer_input, attention_weights)


pred = model.predict(x_test[:10, :, :])
at = a_model.predict(x_test[:10, :, :])
print(np.argmax(pred, axis = 1))
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+ 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(at[i].reshape(28, 1))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
tf = K.tf



class AttentionLayer(Layer):
  def __init__(self, n_cells):
    super(AttentionLayer, self).__init__()
    self.n_cells = n_cells
    
    #weights
    self.w1 = self.add_weight(name = "weight_context",
                             shape = (n_cells, 1),
                             trainable = True,
                             initializer = tf.random_uniform_initializer())
    self.w2 = self.add_weight(name = "weight_hidden",
                             shape = (n_cells, 1),
                             trainable = True,
                             initializer = tf.random_uniform_initializer())
    
  def call(self, inputs):
    hiddens, context = inputs, inputs[-1]
    
    hw = tf.matmul(hiddens, self.w2)
    cw = tf.matmul(context, self.w1)
    m = tf.nn.tanh(hw+cw)
    
    s = tf.nn.softmax(m, axis = 1) #please do not remove axis = 1, check documentation for this function
    e = s*hiddens
    
    z = tf.reduce_sum(e, axis = 1)
    z = tf.expand_dims(z, axis = -1)
    print(s.shape, z.shape)
    
    
    return [z, s]
  
  def compute_output_shape(self, input_shape):
    return [(None, self.n_cells, 1), (None, self.n_cells, 1)]

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 28, 128)           80384     
_________________________________________________________________
activation_1 (Activation)    (None, 28, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
activ

NameError: ignored

# New Section